# Crack Detection using Drone Images

This notebook processes images to detect cracks using various image processing techniques.

In [1]:
!pip install opencv-python-headless pillow matplotlib scikit-image scikit-learn imgaug tensorflow ipywidgets

In [2]:
import cv2
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import numpy as np
from matplotlib import pyplot as plt
import os

In [3]:
# Define the directory containing the images
image_path = os.path.join('c.jpg')  # Replace with your image file name

# Load image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [4]:
# Noise reduction
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

In [5]:
# Adaptive histogram equalization
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced_image = clahe.apply(blurred_image)

In [6]:
# GLCM
glcm = graycomatrix(enhanced_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
contrast = graycoprops(glcm, 'contrast')[0, 0]
dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
energy = graycoprops(glcm, 'energy')[0, 0]
correlation = graycoprops(glcm, 'correlation')[0, 0]
ASM = graycoprops(glcm, 'ASM')[0, 0]

print(f'GLCM - Contrast: {contrast}, Dissimilarity: {dissimilarity}, Homogeneity: {homogeneity}, Energy: {energy}, Correlation: {correlation}, ASM: {ASM}')

In [7]:
# LBP
radius = 1
n_points = 8 * radius
lbp = local_binary_pattern(enhanced_image, n_points, radius, method='uniform')

In [8]:
# Edge detection
edges = cv2.Canny(enhanced_image, 100, 200)

In [9]:
# Display images
plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1), plt.imshow(image, cmap='gray'), plt.title('Original Image')
plt.subplot(2, 2, 2), plt.imshow(blurred_image, cmap='gray'), plt.title('Blurred Image')
plt.subplot(2, 2, 3), plt.imshow(enhanced_image, cmap='gray'), plt.title('Enhanced Image')
plt.subplot(2, 2, 4), plt.imshow(edges, cmap='gray'), plt.title('Edges')
plt.show()

In [10]:
# Feature extraction
features = [contrast, dissimilarity, homogeneity, energy, correlation, ASM]

# Labels (for demonstration, you need actual labels in practice)
labels = np.array([1])  # 1 for crack detected, 0 for no crack

# Split the data
X_train, X_test, y_train, y_test = train_test_split([features], labels, test_size=0.2, random_state=42)

# Train SVM classifier
svm = SVC()
svm.fit(X_train, y_train)

# Predict and evaluate SVM
y_pred_svm = svm.predict(X_test)
print('SVM Classification Report:')
print(classification_report(y_test, y_pred_svm))

# Train MLP classifier
mlp = MLPClassifier(max_iter=1000)
mlp.fit(X_train, y_train)

# Predict and evaluate MLP
y_pred_mlp = mlp.predict(X_test)
print('MLP Classification Report:')
print(classification_report(y_test, y_pred_mlp))

In [11]:
# Crack detection decision based on edge detection
if np.sum(edges) > 0:  # Adjust the condition as needed for accuracy
    print("Crack detected")
else:
    print("No crack detected")